In [16]:
import json
import re
from collections import Counter
from datetime import datetime

from sentence_transformers import SentenceTransformer
import faiss
import numpy as np

In [17]:


class TelegramProAnalyzer:
    def __init__(self):
        self.messages_dict = {}  
        self.users_stats = {}    
        self.interactions = {}   
        self.all_messages_list = []
        # مصفوفة التوقيت (لتحليل الخمول والنشاط)
        self.hourly_activity = Counter() 
        # حساباتك المدمجة
        self.my_accounts = ["لؤي الرازي", "الُحـ๛ـــــآمٌ", "قناة الُحـ๛ـــــآمٌ الرئيسية"]

    def load_and_clean(self, file_paths):
        for file_path in file_paths:
            try:
                with open(file_path, 'r', encoding='utf-8') as f:
                    data = json.load(f)
                    for msg in data.get('messages', []):
                        self.messages_dict[msg.get('id')] = msg
                        self.all_messages_list.append(msg)
                print(f"✅ تم تحميل: {file_path}")
            except Exception as e:
                print(f"❌ خطأ في التحميل: {e}")

    def _extract_text(self, msg):
        text_data = msg.get('text', '')
        if isinstance(text_data, list):
            return "".join([p if isinstance(p, str) else p.get('text', '') for p in text_data])
        return str(text_data)

    def process_data(self):
        self.all_messages_list.sort(key=lambda x: x.get('date', ''))
        
        for msg in self.all_messages_list:
            raw_name = msg.get('from')
            sender_name = str(raw_name) if raw_name is not None else "حساب محذوف"
            
            # توحيد الهوية
            if any(acc in sender_name for acc in self.my_accounts):
                display_name = "أنا (الحسابات المدمجة)"
                sender_id = "unified_me_id"
            else:
                display_name = sender_name
                sender_id = msg.get('from_id', 'unknown_id')

            text = self._extract_text(msg)
            date_str = msg.get('date', '')
            
            # تحليل الوقت (لإجابة سؤال: متى كنا أكثر تواجداً؟)
            if date_str:
                dt = datetime.fromisoformat(date_str)
                self.hourly_activity[dt.hour] += 1

            if sender_id and text.strip():
                if sender_id not in self.users_stats:
                    self.users_stats[sender_id] = {
                        'name': display_name, 'msg_count': 0, 
                        'words_count': 0, 'replies_sent': 0, 
                        'received_replies': 0, 'all_text': "",
                        'initiation_score': 0 # نقاط المبادرة لمشاريع المستقبل
                    }
                
                stats = self.users_stats[sender_id]
                stats['msg_count'] += 1
                stats['words_count'] += len(text.split())
                stats['all_text'] += " " + text.lower()

                # قياس المبادرة (أول رسالة بعد انقطاع طويل تعتبر مبادرة)
                # سنقوم بتطويرها لاحقاً في دوال منفصلة.

                reply_to = msg.get('reply_to_message_id')
                if reply_to:
                    stats['replies_sent'] += 1
                    original_msg = self.messages_dict.get(reply_to)
                    if original_msg:
                        t_raw = original_msg.get('from')
                        t_name = str(t_raw) if t_raw is not None else "حساب محذوف"
                        t_id = "unified_me_id" if any(acc in t_name for acc in self.my_accounts) else original_msg.get('from_id')
                        
                        if t_id in self.users_stats:
                            self.users_stats[t_id]['received_replies'] += 1
                            pair = tuple(sorted([sender_id, t_id])) # توحيد الزوج (أ، ب) هو نفسه (ب، أ)
                            self.interactions[pair] = self.interactions.get(pair, 0) + 1


    def find_best_friends(self):
        """تحليل أقوى الروابط الثنائية بناءً على الردود المتبادلة"""
        if not self.interactions: 
            return "❌ لا توجد بيانات تفاعل كافية."
        
        # ترتيب الأزواج حسب عدد الردود المتبادلة
        sorted_pairs = sorted(self.interactions.items(), key=lambda x: x[1], reverse=True)
        
        results = []
        for pair, count in sorted_pairs[:5]: # نأخذ أعلى 5 علاقات
            name1 = self.users_stats[pair[0]]['name']
            name2 = self.users_stats[pair[1]]['name']
            results.append(f"🤝 {name1} & {name2} | تفاعل: {count} رد متبادل")
        
        return "\n".join(results)      

    def analyze_responsiveness(self):
        """تحديد الشخص الأكثر استجابة وتفاعلاً مع مشاكل الآخرين"""
        # الشخص الخدوم هو الذي يملك أعلى معدل ردود مقارنة بإجمالي رسائله
        responsiveness = []
        for uid, s in self.users_stats.items():
            if s['msg_count'] > 50: # لضمان دقة الإحصاء للأعضاء الفاعلين
                # نسبة الردود التي أرسلها الشخص مقارنة برسائله الكلية
                rate = (s['replies_sent'] / s['msg_count']) * 100
                responsiveness.append((s['name'], rate, s['replies_sent']))
        
        # الترتيب من الأكثر استجابة
        sorted_res = sorted(responsiveness, key=lambda x: x[1], reverse=True)
        
        results = [f"🏆 الأكثر استجابة (الخدوم): {sorted_res[0][0]} بنسبة {sorted_res[0][1]:.1f}%"]
        results.append(f"🥈 المركز الثاني: {sorted_res[1][0]} بنسبة {sorted_res[1][1]:.1f}%")
        return "\n".join(results)

    def get_peak_activity_time(self):
        """تحديد وقت الذروة للمجموعة (متى كنا أكثر تواجداً؟)"""
        if not self.hourly_activity:
            return "❌ لا توجد بيانات زمنية."
        
        # الحصول على الساعة الأكثر نشاطاً
        peak_hour = self.hourly_activity.most_common(1)[0][0]
        
        # تحويل الساعة إلى تنسيق مفهوم (صباحاً/مساءً)
        period = "صباحاً" if peak_hour < 12 else "مساءً"
        display_hour = peak_hour if peak_hour <= 12 else peak_hour - 12
        if display_hour == 0: display_hour = 12
        
        return f"⏰ وقت ذروة المجموعة التاريخي: الساعة {display_hour} {period}" 

    def analyze_personality_traits(self):
        """كشف الشخصية العصبية والشخصية المبادرة"""
        traits_report = []
        
        for uid, s in self.users_stats.items():
            if s['msg_count'] < 20: continue
            
            # 1. حساب "معامل العصبية" (Intensity Score)
            # يعتمد على تكرار علامات التعجب والرسائل المقتضبة جداً
            exclamation_count = s['all_text'].count('!')
            short_msg_ratio = s['words_count'] / s['msg_count']
            
            # معادلة تقريبية للحدة: علامات تعجب كثيرة مع كلام مقتضب
            anger_score = (exclamation_count / s['msg_count']) * 10
            
            # 2. حساب "معامل المبادرة" (Initiative Score)
            # يعتمد على عدد المرات التي بدأ فيها الشخص موضوعاً جديداً (ليس رداً)
            initiation_rate = ((s['msg_count'] - s['replies_sent']) / s['msg_count']) * 100

            # تصنيف الشخصية
            label = "متزن"
            if anger_score > 1.5: label = "عصبي/حاد الطباع"
            elif initiation_rate > 70: label = "قيادي/مبادر"
            elif short_msg_ratio > 15: label = "حكيم/مفصل"
            
            traits_report.append(f"👤 {s['name']}\n   🎭 النمط الغالب: {label}\n   ⚡ معامل الحدة: {anger_score:.2f}")

        return "\n".join(traits_report)         


    def detect_intense_moments(self):
        """كشف اللحظات الحادة مع استخراج نصوص حقيقية لفهم السياق"""
        if len(self.all_messages_list) < 2: return "بيانات غير كافية"

        bursts = []
        current_burst = []
        
        for i in range(1, len(self.all_messages_list)):
            m1 = self.all_messages_list[i-1]
            m2 = self.all_messages_list[i]
            
            try:
                t1 = datetime.fromisoformat(m1.get('date', ''))
                t2 = datetime.fromisoformat(m2.get('date', ''))
                diff = (t2 - t1).total_seconds() / 60
                
                if diff < 3: # رفعنا المدة قليلاً لجمع سياق أكبر
                    current_burst.append(m2)
                else:
                    if len(current_burst) > 30:
                        bursts.append(current_burst)
                    current_burst = []
            except: continue

        top_bursts = sorted(bursts, key=len, reverse=True)[:3]
        
        report = ["📜 سجل اللحظات المفصلية في المجموعة:"]
        for b in top_bursts:
            date = b[0].get('date', '')[:10]
            # بدلاً من الكلمات المفردة، سنأخذ أطول 3 رسائل في هذا الحوار
            # لأن الرسائل الطويلة غالباً هي التي تشرح "أصل الموضوع"
            sorted_messages = sorted(b, key=lambda x: len(self._extract_text(x)), reverse=True)
            context_samples = []
            for m in sorted_messages[:3]:
                sender = m.get('from', 'Unknown')
                text = self._extract_text(m).strip()
                if text: context_samples.append(f"- {sender}: {text[:80]}...")

            report.append(f"\n📅 التاريخ: {date} ({len(b)} رسالة متلاحقة)")
            report.append("🔍 ملخص ما كان يدور:")
            report.extend(context_samples)
            report.append("-" * 30)
            
        return "\n".join(report)  

    def predict_project_roles(self):
        """التنبؤ بأدوار الأعضاء في أي مشروع مستقبلي بناءً على سلوكهم التاريخي"""
        roles_report = ["🔮 تحليل التنبؤ بمبادرات المشاريع القادمة:"]
        
        # كلمات مفتاحية لكل دور
        keywords = {
            'initiator': ['خلونا', 'شرايكم', 'فكرة', 'نعمل', 'نبدأ', 'ايش رايكم'],
            'executor': ['تم', 'جاهز', 'عدلته', 'خلصت', 'ارسلته', 'الكود'],
            'supervisor': ['تمام', 'واصل', 'يا شباب', 'يرجى', 'مهم', 'تنبيه'],
        }

        for uid, s in self.users_stats.items():
            if s['msg_count'] < 50: continue
            
            text = s['all_text']
            scores = {role: 0 for role in keywords}
            
            for role, words in keywords.items():
                for word in words:
                    scores[role] += text.count(word)

            # تحديد الدور الغالب
            main_role = max(scores, key=scores.get)
            
            # حساب "معامل الالتزام" (كم مرة قال "تم" أو "خلصت" مقابل عدد رسائله)
            execution_power = (scores['executor'] / s['msg_count']) * 100
            
            name = s['name']
            if main_role == 'initiator':
                role_desc = "🚀 المبادر (أول من يقترح ويبدأ)"
            elif main_role == 'supervisor':
                role_desc = "⚖️ المشرف (يتابع التنفيذ ويوجه البقية)"
            else:
                role_desc = "🛠️ المنفذ (الذي يحول الأفكار إلى واقع)"

            # التنبؤ بالاستمرارية (من سيتخلى؟)
            # الشخص الذي يختفي فجأة (رسائله قليلة مقارنة بتلقيه للردود)
            is_quitter = "✅ مستمر للنهاية" if s['msg_count'] > 100 else "⚠️ قد ينشغل/يتخلى"

            roles_report.append(f"\n👤 {name}:")
            roles_report.append(f"   ◈ الدور المتوقع: {role_desc}")
            roles_report.append(f"   ◈ قوة التنفيذ: {execution_power:.1f}%")
            roles_report.append(f"   ◈ التنبؤ بالالتزام: {is_quitter}")

        return "\n".join(roles_report)      
    
    def analyze_group_dormancy(self):
        """تحليل أسباب خمول المجموعة وتحديد فترات الانقطاع"""
        if not self.all_messages_list: return "لا توجد بيانات"
        
        yearly_activity = Counter()
        gaps = [] # لتخزين الفجوات الزمنية بالأيام
        
        # 1. توزيع النشاط عبر السنوات
        for i in range(1, len(self.all_messages_list)):
            m1 = self.all_messages_list[i-1]
            m2 = self.all_messages_list[i]
            
            d1 = datetime.fromisoformat(m1.get('date', ''))
            d2 = datetime.fromisoformat(m2.get('date', ''))
            
            yearly_activity[d1.year] += 1
            
            # حساب الفجوة بالأيام
            gap = (d2 - d1).days
            if gap > 7: # إذا انقطعت المجموعة أكثر من أسبوع
                gaps.append({
                    'date': d1.strftime('%Y-%m'),
                    'duration': gap,
                    'last_words': self._extract_text(m1)[:50]
                })

        # 2. صياغة التقرير
        report = ["📉 تحليل ظاهرة الخمول في المجموعة:"]
        
        report.append("\n📊 حجم الرسائل سنوياً:")
        for year in sorted(yearly_activity.keys()):
            report.append(f"   - عام {year}: {yearly_activity[year]} رسالة")
            
        report.append("\n⚠️ أطول فترات الانقطاع وما قيل قبلها:")
        # نأخذ أطول 3 فترات صمت
        sorted_gaps = sorted(gaps, key=lambda x: x['duration'], reverse=True)[:3]
        for g in sorted_gaps:
            report.append(f"   - انقطاع لمدة {g['duration']} يوم (بدأ في {g['date']})")
            report.append(f"     آخر ما قيل: \"{g['last_words']}...\"")
            
        # 3. الاستنتاج الإحصائي
        if yearly_activity[2024] < (yearly_activity.get(2021, 0) / 2):
            report.append("\n💡 الاستنتاج: الخمول ناتج عن تباعد فترات الرد (Time Gaps).")
            report.append("   السبب المرجح: انشغال الأعضاء بالواقع بعد انتهاء ذروة الدراسة.")
            
        return "\n".join(report)
    
    def analyze_user_utility(self):
        """قياس مقدار الفائدة العلمية والمعلوماتية لكل عضو"""
        utility_report = ["💎 تحليل القيمة المضافة (من الأكثر إفادة؟):"]
        
        for uid, s in self.users_stats.items():
            if s['msg_count'] < 30: continue
            
            # 1. حساب نقاط الروابط والملفات (مؤشر فائدة عالي)
            # سنبحث عن علامات الروابط والملفات في النص الشامل لكل عضو
            link_count = s['all_text'].count('http') + s['all_text'].count('www')
            
            # 2. حساب "كثافة المعلومات" (متوسط طول الكلمات في الرسالة)
            info_density = s['words_count'] / s['msg_count']
            
            # 3. معادلة القيمة: (الروابط * 10) + (كثافة المعلومات * 2)
            utility_score = (link_count * 10) + (info_density * 2)
            
            name = s['name']
            
            # تصنيف نوع الفائدة
            if link_count > 15:
                utility_type = "📚 مرجع (يشارك روابط ومصادر)"
            elif info_density > 20:
                utility_type = "🧠 مفكر (رسائله طويلة وشاملة)"
            else:
                utility_type = "💬 اجتماعي (يركز على النقاشات)"
                
            utility_report.append(f"\n👤 {name}:")
            utility_report.append(f"   ◈ التصنيف: {utility_type}")
            utility_report.append(f"   ◈ عدد المصادر المشاركة: {link_count}")
            utility_report.append(f"   ◈ معامل الثقل المعرفي: {utility_score:.1f}")

        return "\n".join(utility_report)
    
    def generate_user_persona(self, target_id):
        """إنشاء بروفايل وصفي شامل لكل عضو بناءً على كافة المحركات"""
        s = self.users_stats.get(target_id)
        if not s: return "العضو غير موجود"

        # 1. استحضار الأرقام الأساسية
        name = s['name']
        activity_level = "عالي جداً" if s['msg_count'] > 3000 else "متوسط"
        
        # 2. تحديد الدور (مبادر/منفذ) وسرعة الاستجابة
        # أضفنا شرطاً بسيطاً للذكاء المحلي
        response_rate = (s['replies_sent'] / s['msg_count']) * 100
        style = "مقتضب وحاسم" if (s['words_count']/s['msg_count']) < 5 else "تفصيلي وشرح"

        # 3. صياغة النبذة (Template-based Profile)
        persona = [
            f"👤 البطاقة الشخصية لـ: {name}",
            f"   • النمط اللغوي: {style}",
            f"   • معدل النشاط: {activity_level}",
            f"   • مستوى التفاعل الاجتماعي: {response_rate:.1f}%",
            f"   • الكلمات الأكثر استخداماً: {s['all_text'].split()[:5]}..."
        ]
        
        return "\n".join(persona)
    

    def ask_the_data(self, query):
        """محرك الإجابة على الأسئلة الحرة (البحث الدلالي المبسط)"""
        results = []
        # تنظيف السؤال من الكلمات الزائدة
        keywords = [w for w in query.split() if len(w) > 3]
        
        for msg in self.all_messages_list:
            text = self._extract_text(msg)
            # إذا وجدت الكلمة المفتاحية في الرسالة
            if any(key in text for key in keywords):
                date = msg.get('date', '')[:10]
                sender = msg.get('from', 'Unknown')
                results.append(f"📅 {date} | {sender}: {text}")
                if len(results) > 5: break # نكتفي بأول 5 نتائج دقيقة
        
        if not results:
            return "🤷‍♂️ عذراً، لم أجد هذا الموقف في ذاكرة المجموعة."
        
        return "\n".join(results)
    
    



In [18]:
class SemanticBrain:
    def __init__(self):
        # تحميل نموذج يدعم العربية وصغير بما يكفي ليعمل محلياً بسرعة
        print("⏳ جاري تحميل عقل اللغة (MiniLM-Arabic)...")
        self.model = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')
        self.index = None
        self.metadata = [] # لتخزين النصوص الأصلية

    def build_memory(self, messages):
        """تحويل الرسائل إلى متجهات وتخزينها في FAISS"""
        print(f"🧠 جاري معالجة {len(messages)} رسالة دلالياً... (قد يستغرق دقائق)")
        
        # تحويل النصوص إلى أرقام
        embeddings = self.model.encode(messages, show_progress_bar=True)
        
        # إنشاء فهرس البحث السريع (FAISS)
        dimension = embeddings.shape[1]
        self.index = faiss.IndexFlatL2(dimension)
        self.index.add(np.array(embeddings).astype('float32'))
        self.metadata = messages
        print("✅ تمت العملية! أصبح البرنامج يفهم معاني رسائلكم الآن.")

    def search(self, query, k=5):
        """البحث عن إجابة بناءً على المعنى"""
        query_vector = self.model.encode([query]).astype('float32')
        distances, indices = self.index.search(query_vector, k)
        return [self.metadata[i] for i in indices[0]]

In [19]:
engine = TelegramProAnalyzer()
engine.load_and_clean(['result1.json', 'result2.json']) 
engine.process_data()
print("🏆 الثنائيات الأكثر تفاهماً:")
print(engine.find_best_friends())

# بعد معالجة البيانات
engine.process_data()

print("-" * 30)
print("📊 تحليل الاستجابة (من الشخص الخدوم؟):")
print(engine.analyze_responsiveness())

print("-" * 30)
print("🕒 تحليل النشاط:")
print(engine.get_peak_activity_time())

# تنفيذ التحليل
print("-" * 30)
print("🧠 تحليل الشخصيات (العصبية والمبادرة):")
print(engine.analyze_personality_traits())

# اكثر اللحظات تفاعلا
print("-" * 30)
print(engine.detect_intense_moments())


# التنبؤ في حال هناك مشروع جديد
print("-" * 30)
print(engine.predict_project_roles())

# سبب خمول المجموعه
print("-" * 30)
print(engine.analyze_group_dormancy())

# دور كل شخصفي المجموعة

print("\n" + "="*40)
print(engine.analyze_user_utility())

# 1. عرض بروفايل صديق معين
print(engine.generate_user_persona('unified_me_id')) # بروفايلك أنت

print("-" * 30)

# 2. اسأل المجموعة أي سؤال (مثلاً عن "تخرج" أو "اختبار" أو "مشروع")
question = "متى تحدثنا عن مشروع التخرج؟"
print(f"🔍 البحث عن: {question}")
print(engine.ask_the_data(question))

✅ تم تحميل: result1.json
✅ تم تحميل: result2.json
🏆 الثنائيات الأكثر تفاهماً:
🤝 م/حسام حسام المياح & المهندس الغالي احمد حاتم | تفاعل: 1100 رد متبادل
🤝 م/ محمد النهاري & م/حسام حسام المياح | تفاعل: 978 رد متبادل
🤝 م/ محمد النهاري & المهندس الغالي احمد حاتم | تفاعل: 967 رد متبادل
🤝 المهندس الغالي/ محمد عبدالسلام & م/حسام حسام المياح | تفاعل: 473 رد متبادل
🤝 م/حسام حسام المياح & م/محمد مطهر زياد | تفاعل: 391 رد متبادل
------------------------------
📊 تحليل الاستجابة (من الشخص الخدوم؟):
🏆 الأكثر استجابة (الخدوم): م/محمد مطهر زياد بنسبة 69.8%
🥈 المركز الثاني: سلطان المعرشي بنسبة 64.3%
------------------------------
🕒 تحليل النشاط:
⏰ وقت ذروة المجموعة التاريخي: الساعة 9 مساءً
------------------------------
🧠 تحليل الشخصيات (العصبية والمبادرة):
👤 م/حسام حسام المياح
   🎭 النمط الغالب: متزن
   ⚡ معامل الحدة: 0.02
👤 المهندس الغالي احمد حاتم
   🎭 النمط الغالب: متزن
   ⚡ معامل الحدة: 0.08
👤 أنا (الحسابات المدمجة)
   🎭 النمط الغالب: قيادي/مبادر
   ⚡ معامل الحدة: 0.09
👤 المهندس الغالي/ محمد عبدالسل

In [20]:
# 1. استخراج الرسائل الطويلة فقط (التي تحتوي على سياق)
rich_messages = [engine._extract_text(m) for m in engine.all_messages_list if len(engine._extract_text(m)) > 15]

# 2. تشغيل العقل الدلالي
brain = SemanticBrain()
brain.build_memory(rich_messages)

# 3. اختبار البحث بالمعنى (بدون كلمات مفتاحية)
print("\n🔍 تجربة البحث بالمعنى:")
# query = "شخص كان دائماً يحفزنا على المذاكرة"
query = "شخص كان دائماً عصبي"
results = brain.search(query)
for r in results:
    print(f"📍 موقف مشابه: {r[:100]}...")
    

⏳ جاري تحميل عقل اللغة (MiniLM-Arabic)...


'(ReadTimeoutError("HTTPSConnectionPool(host='huggingface.co', port=443): Read timed out. (read timeout=10)"), '(Request ID: fd3eb159-9487-4662-9482-8778975e43ba)')' thrown while requesting HEAD https://huggingface.co/api/resolve-cache/models/sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2/86741b4e3f5cb7765a600d3a3d55a0f6a6cb443d/modules.json
Retrying in 1s [Retry 1/5].
'(MaxRetryError('HTTPSConnectionPool(host=\'huggingface.co\', port=443): Max retries exceeded with url: /api/resolve-cache/models/sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2/86741b4e3f5cb7765a600d3a3d55a0f6a6cb443d/modules.json (Caused by NameResolutionError("HTTPSConnection(host=\'huggingface.co\', port=443): Failed to resolve \'huggingface.co\' ([Errno 11002] getaddrinfo failed)"))'), '(Request ID: a18b7b4d-1649-4bd5-af6e-44702667860b)')' thrown while requesting HEAD https://huggingface.co/api/resolve-cache/models/sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2/86741b4e3f5cb7765

🧠 جاري معالجة 11515 رسالة دلالياً... (قد يستغرق دقائق)


Batches:   0%|          | 0/360 [00:00<?, ?it/s]

✅ تمت العملية! أصبح البرنامج يفهم معاني رسائلكم الآن.

🔍 تجربة البحث بالمعنى:
📍 موقف مشابه: لكن يبقى النهاري دائما شخص اناني وكذاب...
📍 موقف مشابه: تفكيره منحرف دائما 


ما يقدر يفكر بشكل مستوي...
📍 موقف مشابه: الصله تمتد إلى الشروووق دائما 


يعني كل موضوع لازم يكون له صله فيها...
📍 موقف مشابه: شي طبيعي بسأل كل واحد يدخل وبنفس الوقت اعمل يدي على صدره اتحسس اضطراب القلب🤤...
📍 موقف مشابه: هو هذا اللي خربط عقلي...
